In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
import sys
sys.path.append("../../")

In [6]:
import tensorflow as tf
from tensorflow import keras

import numpy as np
import os

from tdnn_model import make_tdnn_model
from data_loader import generate_voxc1_ds
from tdnn_model import tdnn_config
print(tf.__version__)

1.14.1-dev20190622


W0702 21:20:50.575076 140400785413888 deprecation_wrapper.py:118] From /home/muncok/anaconda3/envs/tf_nightly/lib/python3.7/site-packages/tensorflow_estimator/python/estimator/api/_v1/estimator/__init__.py:10: The name tf.estimator.inputs is deprecated. Please use tf.compat.v1.estimator.inputs instead.



In [82]:
def quantize(detail, data):
    shape = detail['shape']
    dtype = detail['dtype']
    a, b = detail['quantization']
    
    return (data/a + b).astype(dtype).reshape(shape)


def dequantize(detail, data):
    a, b = detail['quantization']
    
    return (data - b)*a

In [83]:
os.environ['CUDA_VISIBLE_DEVICES']='1'

In [84]:
tf.enable_eager_execution()

In [85]:
from tdnn_model import make_quant_tdnn_model

In [86]:
# train parameters
frame_range = (200, 400)
n_epochs = 60
batch_size = 64
step_per_epoch_ = 134000 // batch_size

# datasets
voxc1_train_dir = "../sv_set/voxc1/fbank64/dev/train/"
voxc1_val_dir = "../sv_set/voxc1/fbank64/dev/val/"
voxc1_test_dir = "../sv_set/voxc1/fbank64/dev/test/"

In [87]:
# include the epoch in the file name. (uses `str.format`)
checkpoint_path = "training_0/tdnn-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path, verbose=1, save_weights_only=True,
    # Save weights, every 5-epochs.
    period=3)

def scheduler(epoch):
    if epoch < 35:
        return 0.001
    elif epoch < 50:
        return 0.0005
    else:
        return 0.0005/2

lr_callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

W0702 22:54:29.316166 140400785413888 callbacks.py:861] `period` argument is deprecated. Please use `save_freq` to specify the frequency in number of samples seen.


In [97]:
steps_per_epoch = 10
# train_x = np.expand_dims(np.load("../../sv_set/voxc1/fbank64/dev/merged/train_500_1.npy"), 2)
# train_y = np.load("../../sv_set/voxc1/fbank64/dev/merged/train_500_1_label.npy")
val_x = np.load("../../sv_set/voxc1/fbank64/dev/merged/val_500_1.npy")
val_y = np.load("../../sv_set/voxc1/fbank64/dev/merged/val_500_label.npy")

def train_generator():
    for x, y in zip(train_x, train_y):
        yield x, y

def val_generator():
    for x, y in zip(val_x, val_y):
        yield x, y

In [15]:
# train
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_graph = tf.Graph()
train_sess = tf.Session(graph=train_graph)

keras.backend.set_session(train_sess)
with train_graph.as_default():
    config = tdnn_config('S')
    train_model = make_quant_tdnn_model(config, 1211, n_frames=500)
    train_ds = tf.data.Dataset.from_generator(train_generator,
        output_types=(tf.float32, tf.int32),
        output_shapes=((500, 1, 65), ()))
    train_ds = train_ds.shuffle(buffer_size=134000)
    train_ds = train_ds.repeat()
    train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
    train_ds = train_ds.batch(batch_size)
    iterator = train_ds.make_one_shot_iterator()
    feat, label = iterator.get_next()
    feat = tf.fake_quant_with_min_max_args(feat, min=-24, max=16)
    tf.contrib.quantize.create_training_graph(input_graph=train_graph, quant_delay=100)
    train_sess.run(tf.global_variables_initializer())    

    train_model.compile(
        optimizer=tf.keras.optimizers.Adam(0.001),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    train_model.fit(feat, label, epochs=1, steps_per_epoch=1000)
    
    # save graph and checkpoints
    saver = tf.train.Saver()
    saver.save(train_sess, 'checkpoints_tdnn')

Train on 1000 samples
 120/1000 [==>...........................] - ETA: 5:29 - loss: 6.7214 - acc: 0.0182

KeyboardInterrupt: 

In [93]:
keras.backend.set_session(train_sess)
with train_graph.as_default():
    print('sample result of original model')
    a = train_model.predict(val_x[4:5])
print(a)
print(a.argmax(), a[0][a.argmax()])
print(val_y[4:5])

NameError: name 'train_sess' is not defined

In [100]:
# eval
eval_graph = tf.Graph()
eval_sess = tf.Session(graph=eval_graph)

keras.backend.set_session(eval_sess)

with eval_graph.as_default():
    keras.backend.set_learning_phase(0)
    config = tdnn_config('S')
    eval_model = make_quant_tdnn_model(config, 1211, n_frames=500)
    tf.contrib.quantize.create_eval_graph(input_graph=eval_graph)
    eval_graph_def = eval_graph.as_graph_def()
    saver = tf.train.Saver()
    saver.restore(eval_sess, 'checkpoints_tdnn')

    frozen_graph_def = tf.graph_util.convert_variables_to_constants(
        eval_sess,
        eval_graph_def,
        [eval_model.output.op.name]
    )

    with open('/tmp/training_quant3/frozen_model.pb', 'wb') as f:
        f.write(frozen_graph_def.SerializeToString())

In [104]:
keras.backend.set_session(eval_sess)
with eval_graph.as_default():
    print('sample result of original model')
    a = eval_model.predict(val_x[7:8])
print(a)
print(a.argmax(), a[0][a.argmax()])
print(val_y[5:6])

sample result of original model
[[0.00082476 0.00109421 0.00056575 ... 0.00042643 0.00062165 0.00090626]]
1006 0.003234057
[0]


In [96]:
val_x[4:5]

array([[[[ 0.14912257,  0.691155  ,  0.7076067 , ..., -0.5425768 ,
          -0.30264288,  0.70040566]],

        [[ 0.06410822,  0.10014683,  0.45650992, ..., -0.63733006,
          -0.56639105, -0.1009497 ]],

        [[-0.4743077 , -0.19535774,  0.33096156, ..., -1.6606073 ,
          -1.5271854 , -1.7403015 ]],

        ...,

        [[ 0.        ,  0.        ,  0.        , ...,  0.        ,
           0.        ,  0.        ]],

        [[ 0.        ,  0.        ,  0.        , ...,  0.        ,
           0.        ,  0.        ]],

        [[ 0.        ,  0.        ,  0.        , ...,  0.        ,
           0.        ,  0.        ]]]], dtype=float32)

### TFLite w/o qunatization

In [78]:
%%bash

tflite_convert \
    --output_file=model.tflite \
    --graph_def_file=/tmp/training_quant3/frozen_model.pb\
    --inference_type=FLOAT \
    --inference_input_type=FLOAT \
    --input_arrays=conv2d_input \
    --input_shape=1,500,1,65 \
    --output_arrays=dense_1/Softmax

2019-07-02 22:53:25.377755: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2019-07-02 22:53:25.387092: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: GeForce GTX TITAN X major: 5 minor: 2 memoryClockRate(GHz): 1.076
pciBusID: 0000:02:00.0
2019-07-02 22:53:25.388268: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 1 with properties: 
name: GeForce GTX TITAN X major: 5 minor: 2 memoryClockRate(GHz): 1.076
pciBusID: 0000:03:00.0
2019-07-02 22:53:25.388393: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.0
2019-07-02 22:53:25.389185: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10.0
2019-07-02 22:53:25.389948: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.1

In [79]:
# load TFLite file
interpreter = tf.lite.Interpreter(model_path=f'model.tflite')
# Allocate memory. 
interpreter.allocate_tensors()

# get some informations .
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print(input_details)
print(output_details)

[{'name': 'conv2d_input', 'index': 62, 'shape': array([  1, 500,   1,  65], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0)}]
[{'name': 'dense_1/Softmax', 'index': 69, 'shape': array([   1, 1211], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0)}]


In [80]:
input_ = val_x[4:5].astype(np.float32)
interpreter.set_tensor(input_details[0]['index'], input_)
interpreter.invoke()

# The results are stored on 'index' of output_details
output_ = interpreter.get_tensor(output_details[0]['index'])

print(output_)
print(output_[0].argmax(), output_[0][output_.argmax()])
print(val_y[16:17])

[[9.3754641e-14 5.5334464e-19 9.1809444e-15 ... 3.0619186e-17
  8.7899775e-07 7.1162509e-07]]
122 0.99682105
[2]


### TFLite w/ qunatization

In [44]:
%%bash

tflite_convert \
    --output_file=model_quant.tflite \
    --graph_def_file=/tmp/training_quant3/frozen_model.pb \
    --inference_type=QUANTIZED_UINT8 \
    --inference_input_type=QUANTIZED_UINT8 \
    --input_arrays=conv2d_input \
    --input_shape=1,500,1,65 \
    --output_arrays=dense_1/Softmax \
    --mean_values=0 \
    --std_dev_values=255

2019-07-02 22:50:01.641740: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2019-07-02 22:50:02.205542: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: GeForce GTX TITAN X major: 5 minor: 2 memoryClockRate(GHz): 1.076
pciBusID: 0000:02:00.0
2019-07-02 22:50:02.206602: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 1 with properties: 
name: GeForce GTX TITAN X major: 5 minor: 2 memoryClockRate(GHz): 1.076
pciBusID: 0000:03:00.0
2019-07-02 22:50:02.206793: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.0
2019-07-02 22:50:02.207883: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10.0
2019-07-02 22:50:02.208824: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.1

In [65]:
# load TFLite file
interpreter = tf.lite.Interpreter(model_path=f'model_quant.tflite')
# Allocate memory. 
interpreter.allocate_tensors()

# get some informations .
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print(input_details)
print(output_details)

[{'name': 'conv2d_input', 'index': 43, 'shape': array([  1, 500,   1,  65], dtype=int32), 'dtype': <class 'numpy.uint8'>, 'quantization': (0.003921568859368563, 0)}]
[{'name': 'dense_1/Softmax', 'index': 49, 'shape': array([   1, 1211], dtype=int32), 'dtype': <class 'numpy.uint8'>, 'quantization': (0.00390625, 0)}]


In [71]:
quantized_input = quantize(input_details[0], val_x[1:2])
# quantized_input = np.round(255 * (val_x[110:111] - features_min) / (features_max - features_min)).astype(np.uint8)
interpreter.set_tensor(input_details[0]['index'], quantized_input)
interpreter.invoke()

# The results are stored on 'index' of output_details
quantized_output = interpreter.get_tensor(output_details[0]['index'])

print('sample result of quantized model')
output_ = dequantize(output_details[0], quantized_output)
print(output_)
print(output_.argmax(), output_[0][output_.argmax()])
print(val_y[4:5])

sample result of quantized model
[[0. 0. 0. ... 0. 0. 0.]]
122 0.98828125
[0]


In [72]:
quantized_input

array([[[[161, 163, 170, ..., 153, 151, 154]],

        [[163, 163, 169, ..., 151, 146, 149]],

        [[164, 159, 167, ..., 145, 145, 149]],

        ...,

        [[172, 168, 164, ..., 164, 175, 177]],

        [[170, 167, 166, ..., 157, 165, 169]],

        [[165, 164, 164, ..., 139, 147, 152]]]], dtype=uint8)

In [47]:
converter = tf.lite.TFLiteConverter.from_frozen_graph(
    '/tmp/training_quant1/frozen_model.pb', ['conv2d_input'],
    ['dense_1/Softmax'], {'conv2d_input': (1, 500, 1, 65)}
)
# conversion quant_aware model to fully quantized model
converter.inference_type = tf.uint8
converter.inference_input_type = tf.uint8
converter.quantized_input_stats = {'conv2d_input': (0, 1)}
tflite_model = converter.convert()

with open('quant_aware.tflite', 'wb') as f:
    f.write(tflite_model)


In [48]:
# load TFLite file
interpreter = tf.lite.Interpreter(model_path=f'quant_aware.tflite')
# Allocate memory. 
interpreter.allocate_tensors()

# get some informations .
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print(input_details)
print(output_details)

[{'name': 'conv2d_input', 'index': 43, 'shape': array([  1, 500,   1,  65], dtype=int32), 'dtype': <class 'numpy.uint8'>, 'quantization': (1.0, 0)}]
[{'name': 'dense_1/Softmax', 'index': 49, 'shape': array([   1, 1211], dtype=int32), 'dtype': <class 'numpy.uint8'>, 'quantization': (0.00390625, 0)}]


In [41]:
val_x = np.load("../../sv_set/voxc1/fbank64/dev/merged/val_500_1.npy")
val_y = np.load("../../sv_set/voxc1/fbank64/dev/merged/val_500_label.npy")

In [42]:
val_x.shape

(6670, 500, 1, 65)

In [ ]:
val_x.

In [50]:
# quantized_input = quantize(input_details[0], val_x[10:11])
features_min = -24
features_max = 16
quantized_input = np.round(255 * (val_x[10:11] - features_min) / (features_max - features_min)).astype(np.uint8)
interpreter.set_tensor(input_details[0]['index'], quantized_input)
interpreter.invoke()

# The results are stored on 'index' of output_details
quantized_output = interpreter.get_tensor(output_details[0]['index'])

print('sample result of quantized model')
output_ = dequantize(output_details[0], quantized_output)
print(output_)
print(output_.argmax(), output_[0][output_.argmax()])
print(val_y[10:11])

sample result of quantized model
[[0. 0. 0. ... 0. 0. 0.]]
1168 0.99609375
[1]


In [99]:
quantized_input

array([[[[160, 167, 172, ..., 149, 153, 147]],

        [[154, 161, 165, ..., 140, 143, 145]],

        [[146, 151, 150, ..., 141, 151, 150]],

        ...,

        [[153, 153, 153, ..., 153, 153, 153]],

        [[153, 153, 153, ..., 153, 153, 153]],

        [[153, 153, 153, ..., 153, 153, 153]]]], dtype=uint8)

In [15]:
sess = tf.InteractiveSession()

/home/muncok/anaconda3/envs/tf_nightly/lib/python3.7/site-packages/tensorflow_core/python/client/session.py:1733: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [19]:
tf.quantization.fake_quant_with_min_max_args(val_x[:1], min=-24, max=16)

<tf.Tensor: id=11, shape=(1, 500, 1, 65), dtype=float32, numpy=
array([[[[ 2.1960793e+00,  1.2549028e+00,  1.5686360e-01, ...,
          -2.6666660e+00, -2.5098031e+00, -6.2745017e-01]],

        [[ 2.0392165e+00,  1.4117656e+00,  9.4117737e-01, ...,
          -2.1960776e+00, -1.8823521e+00,  8.4936619e-07]],

        [[ 1.8823539e+00,  1.8823539e+00,  7.8431463e-01, ...,
          -2.1960776e+00, -7.8431290e-01,  3.1372637e-01]],

        ...,

        [[ 9.4117737e-01,  3.1372637e-01, -1.4117639e+00, ...,
           7.2156873e+00,  7.6862755e+00,  7.5294127e+00]],

        [[ 1.0980401e+00,  6.2745184e-01, -1.0980384e+00, ...,
           7.5294127e+00,  7.8431382e+00,  7.6862755e+00]],

        [[ 6.2745184e-01, -1.7254894e+00, -1.8823521e+00, ...,
           5.1764717e+00,  6.5882363e+00,  6.4313736e+00]]]],
      dtype=float32)>

In [34]:
quant_train_x = np.load("../../sv_set/voxc1/fbank64/dev/merged/train_500_1_quant.npy")

In [39]:
np.quantile(quant_train_x, 0.1)

146.0

In [40]:
np.quantile(quant_train_x, 0.9)

198.0

In [43]:
quant_train_x[0] / 255zzzzzzz

array([[[0.60392157, 0.64313725, 0.65882353, ..., 0.64705882,
         0.64313725, 0.63529412]],

       [[0.63137255, 0.68627451, 0.67843137, ..., 0.65882353,
         0.67058824, 0.65490196]],

       [[0.6627451 , 0.65490196, 0.69411765, ..., 0.6627451 ,
         0.6745098 , 0.6745098 ]],

       ...,

       [[0.70980392, 0.72156863, 0.72941176, ..., 0.62352941,
         0.68627451, 0.6627451 ]],

       [[0.70588235, 0.64705882, 0.71764706, ..., 0.59215686,
         0.65882353, 0.65882353]],

       [[0.70588235, 0.66666667, 0.71764706, ..., 0.58823529,
         0.6627451 , 0.66666667]]])

In [44]:
quant_train_x[1] / 255

array([[[0.72156863, 0.68627451, 0.69019608, ..., 0.66666667,
         0.69019608, 0.68627451]],

       [[0.72156863, 0.68627451, 0.65490196, ..., 0.69803922,
         0.69411765, 0.72941176]],

       [[0.71372549, 0.70196078, 0.65490196, ..., 0.74901961,
         0.69803922, 0.70980392]],

       ...,

       [[0.63921569, 0.65098039, 0.65490196, ..., 0.62745098,
         0.60392157, 0.60784314]],

       [[0.61960784, 0.66666667, 0.61568627, ..., 0.63529412,
         0.6       , 0.58823529]],

       [[0.60392157, 0.68627451, 0.62352941, ..., 0.63921569,
         0.60392157, 0.59215686]]])

In [17]:
np.save("../../sv_set/voxc1/fbank64/dev/merged/train_500_1_quant.npy", train_x_quant)

In [19]:
np.save("../../sv_set/voxc1/fbank64/dev/merged/val_500_1.npy", val_x_quant)